In [14]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By


# -- API --
import re
import json
import requests
from tqdm import tqdm

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [15]:
data = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220822_shopee/shopee_20220829/link_shop_shopee_recrawl.csv')

In [16]:
shop_links = data.shop_url

print(len(shop_links))

2694


In [19]:
def startCrawl(shop_links, list_product):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")

    for link_shop in tqdm(shop_links):
        try:
            Product_Infos = {}

            driver.get(link_shop)

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(0.25)
            driver.execute_script("window.scrollTo(0, 100);")
            sleep(2.5)

            Product_Infos['shop_id'] = link_shop.split('shop/')[1]
            Product_Infos['shop_url'] = link_shop

            Product_Infos['shop_name'] = driver.find_element(by=By.XPATH, value='//h1[@class="section-seller-overview-horizontal__portrait-name"]').text

            # Product_Infos['active_time'] = driver.find_element(by=By.XPATH, value='//div[@class="section-seller-overview-horizontal__inactive-indicator"]').text

            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="shop-page-shop-description"]')) != 0):
                list_description = ''
                Xpath_descriptions = driver.find_elements(by=By.XPATH, value='//div[@class="shop-page-shop-description"]')

                for xpath in Xpath_descriptions:
                    list_description = list_description + xpath.text

                Product_Infos['description'] = list_description

            sleep(0.5)

            img_shopee_mall = 'https://deo.shopeemobile.com/shopee/shopee-pcmall-live-sg/shopmicrofe/483071c49603aa7163a7f51708bff61b.png'

            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="official-shop-new-badge"]/img')) != 0):
                for shop_type in driver.find_elements(by=By.XPATH, value='//div[@class="official-shop-new-badge"]/img'):
                    if (img_shopee_mall) == (shop_type.get_attribute('src')):
                        Product_Infos['type_shop'] = 'shopee_mall'
            else:
                Product_Infos['type_shop'] = ''

            Xpaths_info_basics = driver.find_elements(by=By.XPATH, value='//div[@class="section-seller-overview__item-text"]')

            for Xpath in Xpaths_info_basics:
                if('Đang Theo:' in Xpath.text):
                    Product_Infos['follower'] =  re.findall('\d+',Xpath.text)[0]
                if('Tỉ Lệ Phản Hồi Chat:' in Xpath.text):
                    Product_Infos['response_rate'] = re.findall('\d+',Xpath.text)[0]
                if('Tham Gia:' in Xpath.text):
                    Product_Infos['joined_time'] = re.findall('\d+',Xpath.text)[0]
                if('Đánh Giá' in Xpath.text):
                    if('(' not in  Xpath.text):
                        Product_Infos['avg_rating'] = re.findall('\d+',Xpath.text)[0]
                        Product_Infos['total_rating_count'] = re.findall('\d+',Xpath.text)[0]
                    elif('(' in  Xpath.text):
                        Product_Infos['avg_rating'] = Xpath.text.split(' (')[0].split(': ')[1]
                        Product_Infos['total_rating_count'] = Xpath.text.split(' (')[1].split(' Đánh Giá')[0]
                if('Sản Phẩm:' in Xpath.text):
                    Product_Infos['num_of_product'] = re.findall('\d+',Xpath.text)[0]

            Product_Infos['platform'] = 'shopee'

            try:
                Product_Infos['location'] = driver.find_element(by=By.XPATH, value='//div[@class="_1IbMik"]').text
            except:
                pass

            list_product.append(Product_Infos)
            sleep(0.5)
        except:
            continue

In [20]:
import threading
import numpy as np

list_threads = []
list_product = []
num_thread = 3
list_partitions = [partition for partition in np.array_split(shop_links, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, list_product)))

In [ ]:
for thread in list_threads:
    thread.start()
    time.sleep(5)

In [ ]:
for thread in list_threads:
    thread.join()

In [ ]:
pd.DataFrame(list_product).sample(3)

In [24]:
len(list_product)

2682

In [ ]:
pd.DataFrame(list_product).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220822_shopee/Shopee_20220825/shopee_shop_info_full_recrawl.csv', index=False)